# 🧮 Torque & Preload Analysis — Active Omni PipeBot
This notebook computes design torques and forces for the two-wheel pipe-inspection robot.

Formulas from `docs/Mechanism_Explanation.md`:

\[
\tau_{wheel} = \frac{m g R_h}{2}, \qquad N = \frac{m g}{2\mu}, \qquad T_{pinion} = N\,\rho_p
\]
with safety factor *SF* (1.5–2×).

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- User parameters (edit as needed) ---
m = 3.5          # robot mass [kg]
mu = 0.6         # friction coefficient wheel–pipe
Rh = 0.025       # wheel radius [m]
rho_p = 0.01     # pinion pitch radius [m]
g = 9.81         # gravity [m/s^2]
SF = 1.5         # safety factor

# --- Calculations ---
N = (m*g)/(2*mu)            # normal force per wheel [N]
tau_wheel = (m*g*Rh)/2      # drive torque per wheel [N·m]
T_pinion = N*rho_p          # pinion torque [N·m]

results = pd.DataFrame({
    'Parameter': ['Normal Force N', 'Wheel Torque τ_wheel', 'Pinion Torque T_pinion'],
    'Nominal (N or N·m)': [N, tau_wheel, T_pinion],
    'With Safety Factor (×SF)': [N*SF, tau_wheel*SF, T_pinion*SF]
})
results

### 1️⃣ Results Table

In [ ]:
display(results.style.format({'Nominal (N or N·m)': '{:.2f}', 'With Safety Factor (×SF)': '{:.2f}'}))

### 2️⃣ Parametric sweep — torque vs. mass and friction

In [ ]:
# Range of parameters
masses = np.linspace(2,5,20)
frictions = np.linspace(0.4,0.8,20)

M, MU = np.meshgrid(masses, frictions)
N_grid = (M*g)/(2*MU)
tau_grid = (M*g*Rh)/2

# Contour plot
plt.figure(figsize=(6,4))
cp = plt.contourf(M, MU, tau_grid*SF, levels=15, cmap='viridis')
plt.colorbar(cp,label='Required Wheel Torque ×SF [N·m]')
plt.xlabel('Robot Mass [kg]')
plt.ylabel('Friction Coefficient μ')
plt.title('Torque Requirement vs Mass & Friction')
plt.tight_layout()
plt.show()

### 3️⃣ Interpretation
- **Higher mass → higher torque and normal load.**
- **Lower μ → more rack preload needed** to maintain traction.
- Motors and pinion drives should be selected for ≥ the values in the last column above.

Save your computed table as CSV:

In [ ]:
results.to_csv('torque_results.csv', index=False)
print('Saved torque_results.csv in analysis folder')

### 4️⃣ Next Steps
- Integrate these results into the Stage 2 report.
- Use output torques for **motor sizing** and **gear-ratio selection**.
- Validate results against **Fusion 360 Motion Study** outputs.
- Commit `torque_results.csv` back to GitHub.